**IMPORTAR LIBRERIAS**

---

In [ ]:
# Librerias de manejo de datos
import pandas as pd

In [ ]:
# Importar conexión
from db_connection import get_connection

# Conectarse a la base de datos
conexion = get_connection()
cursor = conexion.cursor()


In [ ]:
# Importar librerías para la conexión a la base de datos con SQLAlchemy
from db_connection import get_engine

engine = get_engine()

In [ ]:
# 📌 Obtener todas las tablas
cursor.execute("SHOW TABLES;")
tablas = [fila[0] for fila in cursor.fetchall()]
print(f"Total de tablas encontradas: {len(tablas)}")


---

**FUNCIONES**

---

In [ ]:
def borrar_lista_tablas(tablas_a_borrar):
    """
    Borra las tablas de la base de datos que se pasan como argumento.
    """
    for tabla in tablas_a_borrar:
        try:
            cursor.execute(f"DROP TABLE `{tabla}`;")
            tablas.remove(tabla)
            print(f"✅ Tabla eliminada: {tabla}")
        except Exception as e:
            print(f"❌ Error al eliminar {tabla}: {e}")
    
    # Confirmar cambios
    conexion.commit()

------

**FILTROS PARA SEGREGAR Y ELIMINAR LAS TABLAS QUE NO SON ÚTILES**


---

*PRIMER FILTRO: Tablas con menos de 2 registros*

In [ ]:

# Escoger tablas a eliminar
tablas_a_borrar = []

for tabla in tablas:
    try:
        cursor.execute(f"SELECT COUNT(*) FROM `{tabla}`;")
        total = cursor.fetchone()[0]
        if total <= 1:
            tablas_a_borrar.append(tabla)
    except Exception as e:
        print(f"❌ Error evaluando tabla {tabla}: {e}")

print(f"\nTablas candidatas para eliminación ({len(tablas_a_borrar)}):")
print(tablas_a_borrar)


In [ ]:
# ⚠️ Solo si tienes respaldo y estás en entorno seguro
# ELIMINAR TABLAS CANDIDATAS

# Desactivar restricciones
cursor.execute("SET FOREIGN_KEY_CHECKS=0;")

borrar_lista_tablas(tablas_a_borrar)

# Volver a activar restricciones
cursor.execute("SET FOREIGN_KEY_CHECKS=1;")



---

*SEGUNDO FILTRO: Tablas posiblemente innecesarias por el nombre y breve análisis*

In [ ]:
# Imprimir los nombres de las tablas restantes
for tabla in tablas:
    print(tabla)


*ANÁLISIS Y CONCLUSIONES DE LA INFORMACIÓN OBTENIDA*

**Tablas clave para análisis de gestión empresarial (💡 Mantener)**

Estas son probablemente las más útiles para análisis de KPIs, dashboards o visualización de negocio:

*Categoría	//  Tablas clave*

📦 Inventario	= items, item_balance, item_kardex, warehouses, inventory_adjustments, item_categories, item_subcategories, item_variations

💼 Ventas / Documentos	= documents, document_items, contacts (clientes/proveedores)

🧾 Contabilidad	= accounting_vouchers, accounting_voucher_items, accounting_accounts, accounting_movements, retentions, retention_concepts, retentions_applied, accounting_account_balances

👥 Empleados / Nómina	= employees, employee_contracts, payrolls, payroll_details, payroll_incomes, payroll_deductions

🛒 Compras	= ecommerce_purchase_orders

📊 Precios	= price_lists

🏪 Organización	= headquarters, user_headquarters, roles, user_roles, company_areas

🧾 Impuestos	= taxes

🏭 Activos fijos	= fixed_assets, fixed_asset_depreciations, fixed_assets_groups


**Relevancia baja o dependiente del contexto (🤔 Revisar)**

*Estas podrían ser útiles dependiendo del foco del proyecto (clientes, pedidos, legal, e-commerce):*

contact_relationships, contact_accounts, contact_statements

coupons, coupon_groups (si hay ecommerce)

employee_positions, contract_salary_history

plan_electronic_documents, plan_system_controller

ecommerce_legal_info

costs_and_expenses, costs_and_expenses_categories

radian_documents (puede ser de facturación electrónica)

dining_tables (¿restaurante? puede ser irrelevante)

system_counters, system_restrictions

In [ ]:
# Lista de tablas candidatas a eliminar
tablas_candidatas = [
    "api_access_tokens",
    "electronic_payroll_test_set",
    "migrations",
    "oauth_clients",
    "paynilo",
    "role_permissions",
    "template_versions",
    "templates",
    "user_data",
    "values_x_item",
    "warehouse_transfer_logs",
    "billing_numberings",
    "custom_fields",
    "configurations",
    "system_counters",
    "system_restrictions"
]

---

*TERCER FILTRO: tablas a eliminar según sus columnas y número de filas*

In [ ]:

# Mostrar información muy básica de las tablas candidatas
for tabla in tablas_candidatas:
    try:
        df = pd.read_sql(f"SELECT * FROM `{tabla}` LIMIT 5", engine)
        print(f"\n📘 Tabla: {tabla}")
        print("Columnas:", df.columns.tolist())
        print("size: ", df.size)
    except Exception as e:
        print(f"\n❌ Error al cargar tabla {tabla}: {e}")

**Tablas candidatas fuertes para eliminación (por contenido y nombre)**

Estas tablas tienen pocos registros, estructuras irrelevantes para KPIs o funciones accesorias que no impactan directamente en la gestión:

*Tabla	// Motivo principal*

api_access_tokens	Manejo técnico de tokens

electronic_payroll_test_set	Datos de prueba (test set)

migrations	Solo para migraciones del sistema

oauth_clients	Seguridad/autenticación interna

paynilo	Integración de pagos externa

role_permissions	Control de permisos internos

template_versions	Versiones de plantillas, sin datos

templates	Estructura visual, no analítica

user_data	Solo datos de área/cargo del usuario

custom_fields	Campos personalizados (metadatos)

system_counters	Estadísticas internas del sistema

system_restrictions	Restricciones de uso/licencia

values_x_item	Datos muy técnicos por item, posiblemente redundantes

warehouse_transfer_logs	Registro técnico, no necesariamente útil para KPIs

----

*CUARTO FILTRO: Investigar mas a fondo las tablas dudosas*

**⚠️ Tablas dudosas (verificar con .info() antes de decidir)**

Estas tienen algo más de contenido o potencial analítico dependiendo del negocio:

*billing_numberings:* Tiene 155 registros. Podría contener información útil para facturación si los números se asocian a documentos.

*configurations:* Solo 25 registros, pero hay que ver si son configuraciones claves que afectan reportes.



In [ ]:
# Obtener el esquema de cada tabla para decidir si son candidatas a eliminación
tablas_dudosas = ['billing_numberings', 'configurations'] # Agregar mas tablas si es necesario

for tabla in tablas_dudosas:
    print(f"\n📋 Tabla: {tabla}")
    try:
        df = pd.read_sql(f"SELECT * FROM {tabla}", con=engine)
        print("▶️ .info():")
        print(df.info())
        print("▶️ Primeras filas:")
        display(df.head())
    except Exception as e:
        print(f"❌ Error al cargar {tabla}: {e}")


---

**TABLAS FINALES A ELIMINAR DESPUÉS DE TODOS LOS FILTROS Y ANÁLISIS INDIVIDUAL**

Falta escoger y seleccionar individualmente 

In [ ]:
tablas_a_eliminar = [
    "api_access_tokens",             # Manejo técnico de tokens
    "electronic_payroll_test_set",   # Datos de prueba (test set)
    "migrations",                    # Solo para migraciones del sistema
    "oauth_clients",                 # Seguridad/autenticación interna
    "paynilo",                       # Integración de pagos externa
    "role_permissions",             # Control de permisos internos
    "template_versions",            # Versiones de plantillas, sin datos
    "templates",                    # Estructura visual, no analítica
    "user_data",                    # Solo datos de área/cargo del usuario
    "custom_fields",                # Campos personalizados (metadatos)
    "system_counters",              # Estadísticas internas del sistema
    "system_restrictions",          # Restricciones de uso/licencia
    "values_x_item",                # Datos técnicos por ítem
    "warehouse_transfer_logs",      # Registro técnico, no útil para KPIs
]

In [ ]:
# ⚠️ Solo si tienes respaldo y estás en entorno seguro
# ELIMINAR TABLAS FINALES ¡OJO HAY QUE ESCOGER ANTES DE EJECUTAR!

# Desactivar restricciones !NO SE HACE ANTES DE VERIFICAR QUE NO GENERA NINGÚN DAÑO!
##cursor.execute("SET FOREIGN_KEY_CHECKS=0;")

#borrar_lista_tablas(tablas_a_borrar)

# Volver a activar restricciones !NO SE HACE ANTES DE VERIFICAR QUE NO GENERA NINGÚN DAÑO!
##cursor.execute("SET FOREIGN_KEY_CHECKS=1;")
